# Clasificación de texto utilizando LSTM


In [1]:
#Importamos nuestras librerias
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Descargamos los datos
!curl -O http://srodriguez.me/Datasets/imdb.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.7M  100 25.7M    0     0   9.9M      0  0:00:02  0:00:02 --:--:--  9.9M


In [3]:
#Descomprimimos los datos
!unzip imdb.zip

Archive:  imdb.zip
  inflating: IMDB Dataset.csv        


# Carga de datos

En este caso vamos a cargar los datos de IMDB, para clasificación de sentimiento 

In [4]:
#Generamos nuestro Dataframe, leyendo el archivo .csv
df = pd.read_csv("IMDB Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [5]:
# Mostramos los primero 5 elementos
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
#Muestreamos 10.000 reviews en vez de procesar todo el conjunto de dato
df = df.sample(10000)
df.reset_index(drop=True,inplace=True) #Reiniciamos los indices de este nuevo df

In [7]:
# Se transforma el texto de las etiquetas en valores numericos para el proceso de aprendizaje
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [8]:
# Mostramos los primero 5 elementos
df.head()

,review,sentiment
0,This is a rip-off of already crappy hollywood ...,0
1,Will Farmer (Lanter) plays a computer game tha...,0
2,if.... is the cinematic equivalent of Sgt. Pep...,0
3,The Neil Simon's Sunshine Boys starring Walter...,1
4,I own this movie. And it is terribly hard to f...,0


In [9]:
#Vemos el primer review del nuevo set de datos
df.review[0]

"This is a rip-off of already crappy hollywood movies like Scream and I Know What You Did Last Summer. The story is classic, some high-school students tries a prank on the class' asthmatic misfit but something goes wrong. Terribly wrong. When you watch the movie you know what'll happen before it happens all the time, not good if a movie tries to be scary. The actors are quite ok and the girls are cute (after all, they're asian) so i'll give it two out of five on the mojave'o'meter."

In [10]:
#Importamos la funcion para hacer separacion de los distintos conjuntos de entrenamiento
from sklearn.model_selection import train_test_split

In [11]:
#Separamos un 30% de datos para test
temp_df, test_df = train_test_split(df,test_size=0.3,random_state=42)
#y de training, sacamos un 10% para validación
train_df, val_df = train_test_split(temp_df,test_size=0.1,random_state=42)

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [12]:
max_features = 20000  # Considerar las primeras 20000 palabras para generar un diccionario
maxlen = 200  # Considerar las primeras 200 palabras de cada review

#Generamos nuestro tokenizador, el cual nos va a permitir generar nuestro diccionario
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features, oov_token='<unk>')

In [13]:
#Construimos el vocabulario
tokenizer.fit_on_texts(train_df['review'])

In [17]:
#Vemos que palabra corresponde al indice numero 2
tokenizer.index_word[2]

'the'

In [18]:
#Vemos que indice que corresponde la palabra 'the'
tokenizer.word_index['the']

2

In [19]:
#Generamos las secuencias al transformar de texto a valores numericos
secuencias = tokenizer.texts_to_sequences(train_df['review'])


In [20]:
#Fijamos las secuencias en un largo especifico, añadiendo los token de padding '<pad>'
secuencias_padded = keras.preprocessing.sequence.pad_sequences(secuencias,maxlen= maxlen)
secuencias_padded

array([[    0,     0,     0, ...,  1230,  1159,  1175],
       [    0,     0,     0, ...,    23,   543,   202],
       [    0,     0,     0, ...,    92,  7226,     1],
       ...,
       [    2,   697,  2625, ...,    71,   268,   305],
       [    0,     0,     0, ...,   317,   164, 14913],
       [    0,     0,     0, ...,    15,    20,  1279]], dtype=int32)

In [21]:
maxlen

200

In [22]:
# Transformamos el texto a secuencia para los conjuntos de validacion y testing
val_seq = tokenizer.texts_to_sequences(val_df['review'])
val_seq_padded = keras.preprocessing.sequence.pad_sequences(val_seq,maxlen= maxlen)

test_seq = tokenizer.texts_to_sequences(test_df['review'])
test_seq_padded = keras.preprocessing.sequence.pad_sequences(test_seq,maxlen= maxlen)

In [23]:
#Generamos nuestras entradas para la red
X_train = secuencias_padded
y_train =train_df['sentiment']

X_val = val_seq_padded
y_val = val_df['sentiment']

X_test = test_seq_padded
y_test = test_df['sentiment']

In [24]:
#Capa de entrada, la cual recibira los arreglos de enteros (indices del vocabulario)
inputs = keras.Input(shape=(maxlen,), dtype="int32")
# Transformamos cada indice, en su vector de palabras correspondiente
x = layers.Embedding(max_features, 128)(inputs)
#Añadimos una capa de LSTM
x = layers.LSTM(64)(x)

# Añadimos la capa de salida, 1 neurona de salida debido a que es clasificación binaria
# Ademas, utilizamos la funcion de activación sigmoidea para que arroje la probabilidad
outputs = layers.Dense(1, activation="sigmoid")(x)

#Generamos el modelo
model = keras.Model(inputs, outputs)
model.summary() # E imprimimos el modelo 


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 128)          2560000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,609,473
Trainable params: 2,609,473
Non-trainable params: 0
_________________________________________________________________


In [25]:
#Compilamos el modelo, seleccionamos el optimizador, la funcion de perdida y la metrica de exactitud
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#Ajustamos el modelo, utilizando los conjuntos de entrenamiento y validamos con el conjunto de validación
#Entrenamos por 5 "Epocas"
model.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val))


Epoch 1/5
394/394 [==============================] - 25s 45ms/step - loss: 0.5327 - accuracy: 0.7263 - val_loss: 0.4466 - val_accuracy: 0.7900
Epoch 2/5
394/394 [==============================] - 17s 43ms/step - loss: 0.2413 - accuracy: 0.9071 - val_loss: 0.4930 - val_accuracy: 0.7800
Epoch 3/5
394/394 [==============================] - 17s 43ms/step - loss: 0.1092 - accuracy: 0.9635 - val_loss: 0.5512 - val_accuracy: 0.7957
Epoch 4/5
394/394 [==============================] - 17s 43ms/step - loss: 0.0980 - accuracy: 0.9648 - val_loss: 0.6333 - val_accuracy: 0.8143
Epoch 5/5
394/394 [==============================] - 17s 43ms/step - loss: 0.0309 - accuracy: 0.9902 - val_loss: 0.7445 - val_accuracy: 0.8257


In [26]:
#Realizamos la prediccion del modelo y vemos el output (Son probabilidades)
y_pred = model.predict(X_test,batch_size=16,verbose=1)
y_pred

188/188 [==============================] - 3s 12ms/step


array([[0.02583051],
       [0.99830306],
       [0.00155848],
       ...,
       [0.9998217 ],
       [0.9992976 ],
       [0.63899755]], dtype=float32)

In [27]:
#Transformamos la prediccion en valores binarios al preguntar por todos los valores
# mayores o iguales a 0.5
y_pred = y_pred >= 0.5
y_pred

array([[False],
       [ True],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [28]:
#Transformamos los valores binarios a '0' y '1' utilizando una representación de enteros
y_pred = y_pred.astype(np.int8)
y_pred

array([[0],
       [1],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int8)

In [29]:
#Aplanamos el vector para que quede de forma unidimensional
y_pred = y_pred.reshape(-1)
y_pred

array([0, 1, 0, ..., 1, 1, 1], dtype=int8)

In [30]:
#Importamos la libreria de métricas de scikit-learn
from sklearn import metrics

In [31]:
#Imprimimos los valores de prueba, solo por sanity check
print(y_test.values)

[1 1 0 ... 1 1 0]


In [32]:
#Imprimimos las metricas
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.8393333333333334
              precision    recall  f1-score   support

           0       0.83      0.85      0.84      1500
           1       0.85      0.83      0.84      1500

    accuracy                           0.84      3000
   macro avg       0.84      0.84      0.84      3000
weighted avg       0.84      0.84      0.84      3000



# Utilizando vectores de palabras pre-entrenados

https://github.com/RaRe-Technologies/gensim-data <- De aca podemos obtener modelos pre-entrenados soportados por la misma libreria de GenSim

In [33]:
import gensim.downloader as api
#En este caso, vamos a utilizar los vectores entrenados en wikipedia mediante el algoritmo Glove
wv = api.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [34]:
#Verificamos que la palabra 'the' exista y mostramos los primeros 10 valores
wv['the'][:10]

array([ 0.04656  ,  0.21318  , -0.0074364, -0.45854  , -0.035639 ,
        0.23643  , -0.28836  ,  0.21521  , -0.13486  , -1.6413   ],
      dtype=float32)

In [35]:
# Instanciamos una matriz incializada al azar con un vocabulario de max_features + 1
# Esto ultimo es para considerar el vector asociado al token '<pad>'
emb_matrix = np.random.rand(max_features + 1,300) #El largo de nuestro vector

emb_matrix[0][:10]

array([0.43213699, 0.50750642, 0.96434454, 0.13383361, 0.05451138,
       0.49116126, 0.45587156, 0.21525036, 0.26060986, 0.72168505])

In [36]:
#Importamos una libreria para hacer seguimiento del procesos de traspaso de embeddings
from tqdm.notebook import tqdm

In [37]:
#Iteramos por cada una de las palabras
for i in tqdm(range(1, max_features + 1)):
  #Obtenemos la palabra correspondiente al indice i
  word = tokenizer.index_word[i]
  if word in wv: #Preguntamos si la palabra esta en el modelo de vectores de palabra
    emb_matrix[i] = wv[word] # Asignamos el valor a la fila al vector de palabra


  0%|          | 0/20000 [00:00<?, ?it/s]

In [38]:
#Comprobamos que el la matriz de embeddings populada, el indice 2 (para la palabra 'the') 
#corresponde al valor que vimos anteriormente
emb_matrix[2][:10]

array([ 0.04656   ,  0.21318001, -0.0074364 , -0.45853999, -0.035639  ,
        0.23643   , -0.28836   ,  0.21521001, -0.13485999, -1.64129996])

In [42]:
#Capa de entrada, la cual recibira los arreglos de interos (indices del vocabulario)
inputs = keras.Input(shape=(maxlen,), dtype="int32")
# Transformamos cada indice, en su vector de palabras correspondiente
# Pero esta vez, utilizando el parametro weights, inicializamos esta capa de vectores
# Con los pesos extraidos del modelo pre-entrenado
# Aparte, definimos que esta capa no sea entrenable, o sea que los valores de los pesos
# No se vayan ajustando en cada iteración de entrenamiento
x = layers.Embedding(max_features+1, 300,weights=[emb_matrix], trainable=False)(inputs)
#Añadimos una capa de LSTM
x = layers.LSTM(64)(x)
# Añadimos la capa de salida, 1 neurona de salida debido a que es clasificación binaria
# Ademas, utilizamos la funcion de activación sigmoidea para que arroje la probabilidad
outputs = layers.Dense(1, activation="sigmoid")(x)

#Generamos el modelo
model2 = keras.Model(inputs, outputs)
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 200, 300)          6000300   
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6,093,805
Trainable params: 93,505
Non-trainable params: 6,000,300
_________________________________________________________________


In [ ]:
#Evaluamos y obtenemos las metricas

In [43]:
model2.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model2.fit(X_train, y_train, batch_size=16, epochs=5, validation_data=(X_val, y_val))


Epoch 1/5
394/394 [==============================] - 18s 41ms/step - loss: 0.5996 - accuracy: 0.6727 - val_loss: 0.5402 - val_accuracy: 0.7171
Epoch 2/5
394/394 [==============================] - 16s 40ms/step - loss: 0.4320 - accuracy: 0.8022 - val_loss: 0.4648 - val_accuracy: 0.7671
Epoch 3/5
394/394 [==============================] - 16s 40ms/step - loss: 0.3279 - accuracy: 0.8614 - val_loss: 0.3638 - val_accuracy: 0.8457
Epoch 4/5
394/394 [==============================] - 16s 40ms/step - loss: 0.2669 - accuracy: 0.8873 - val_loss: 0.3464 - val_accuracy: 0.8429
Epoch 5/5
394/394 [==============================] - 16s 40ms/step - loss: 0.2151 - accuracy: 0.9127 - val_loss: 0.3681 - val_accuracy: 0.8471


In [44]:
y_pred = model2.predict(X_test,batch_size=16,verbose=1)
y_pred = y_pred >= 0.5
y_pred = y_pred.astype(np.int8).reshape(-1)

188/188 [==============================] - 3s 12ms/step


In [45]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.8596666666666667
              precision    recall  f1-score   support

           0       0.90      0.81      0.85      1500
           1       0.82      0.91      0.87      1500

    accuracy                           0.86      3000
   macro avg       0.86      0.86      0.86      3000
weighted avg       0.86      0.86      0.86      3000

